In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 12.2 MB/s eta 0:00:00


In [3]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

In [4]:
# Función genérica para procesar un archivo TTL y extraer propiedades específicas
def ttl_to_dataframe_generic(ruta_ttl, propiedades_interes, prefijo_id):
    """
    Procesa un archivo TTL y extrae las propiedades indicadas.

    :param ruta_ttl: Ruta del archivo TTL
    :param propiedades_interes: Diccionario con las propiedades que se quieren extraer
    :return: DataFrame con las propiedades extraídas
    """
    # Cargar el archivo TTL con rdflib
    g = Graph()
    g.parse(ruta_ttl, format="ttl")

    # Diccionario para almacenar los datos procesados
    registros = {}

    # Iterar sobre las tripletas y extraer los datos
    for subj, pred, obj in g:
        # Extraer el identificador del sujeto (por ejemplo, cliente44)
        sujeto = str(subj).split('/')[-1]

        # Verificar si el predicado está en las propiedades de interés
        for prop_uri, prop_nombre in propiedades_interes.items():
            if prop_uri in str(pred):
                # Asegurar que el sujeto esté en el diccionario
                if sujeto not in registros:
                    registros[sujeto] = {}
                # Guardar la propiedad mapeada con su nombre definido
                registros[sujeto][prop_nombre] = str(obj)

    # Convertir el diccionario a DataFrame
    df = pd.DataFrame.from_dict(registros, orient="index").reset_index()
    df.rename(columns={"index": f"IDENTIFICADOR_{prefijo_id}"}, inplace=True)
    return df


In [5]:
# Cargar archivo principal
df_consolidado = None
df_consolidado = pd.read_csv("https://deinsoluciones.cl/web_semantica/informacion_base_importaciones_vehiculos.csv", delimiter=",",encoding='utf-8')
df_consolidado.head()

,NUMENCRIPTADO,TPO_DOCTO,NOMBRE_ADUANA,NOMBRE_COMUNA,NUM_UNICO_IMPORTADOR,NOMBRE_PAIS_ORIG,NOMBRE_VIA_TRANSPORTE,NOMBRE_PUERTO_EMBARQUE,NOMBRE_PUERTO_DESEMBARQUE,NOMBRE_TIPO_CARGA,NUM_MANIF,NOMBRE_NAVE,NOMBRE_AGENTE_DE_NAVES,NUM_CONOC,FEC_CONOC,NOMEMISOR,NOMBRE_MONEDA,FOB,FLETE,SEGURO,CIF,ARANC_NAC,TOTAL_BULTOS
0,20955958,151,SAN ANTONIO,LAMPA,12516,CHINA,"MARITIMA, FLUVIAL Y LACUSTRE",OTROS PTOS.DE CHINA,SAN ANTONIO,GENERAL,239233,XIN YA ZHOU,SOCIEDAD MARITIMA Y COMERCIAL SOMARCO LTDA.,(M)COSU6370424530,3122023,MARITRANS,DOLAR USA,309000,25035,340,334375,87024090,4
1,20981434,103,SAN ANTONIO,PUDAHUEL,10870,COREA DEL SUR,"MARITIMA, FLUVIAL Y LACUSTRE",OTROS PTOS. COREA S,SAN ANTONIO,GENERAL,237767,BESS,IAN TAYLOR Y CIA. S.A.,EUKOHUSQ1935013,31102023,EUKOR CAR CARRIERS,DOLAR USA,22437,"1610,02","18,23","24065,25",87021019,1
2,21014856,103,SAN ANTONIO,PUDAHUEL,10870,COREA DEL SUR,"MARITIMA, FLUVIAL Y LACUSTRE",OTROS PTOS. COREA S,SAN ANTONIO,GENERAL,237767,BESS,IAN TAYLOR Y CIA. S.A.,EUKOHUSQ1935017,31102023,EUKOR CAR CARRIERS,DOLAR USA,22437,"1610,02","18,21","24065,23",87021019,1
3,20985879,103,SAN ANTONIO,PUDAHUEL,10870,COREA DEL SUR,"MARITIMA, FLUVIAL Y LACUSTRE",OTROS PTOS. COREA S,SAN ANTONIO,GENERAL,238749,MORNING CHORUS,IAN TAYLOR Y CIA. S.A.,HDGLKRCL0562033,30112023,HYUNDAI GLOVIS,DOLAR USA,22437,"1630,32","18,24","24085,56",87021019,1
4,20993803,151,SAN ANTONIO,PUDAHUEL,10870,COREA DEL SUR,"MARITIMA, FLUVIAL Y LACUSTRE",OTROS PTOS. COREA S,SAN ANTONIO,GENERAL,239633,MORNING CHERRY,IAN TAYLOR Y CIA. S.A.,EUKOHUSQ1944340,28122023,EUKOR CAR CARRIERS,DOLAR USA,292031,"21194,16","237,45","313462,61",87021019,26


In [6]:
# Cargar archivo de aduanas
ruta_ttl_aduana = "https://deinsoluciones.cl/web_semantica/aduanas.ttl"
prefijo = "ADUANA"
propiedades_aduana= {
    "http://schema.org/nombre": "NOMBRE_ADUANA_TTL"
}
df_ttl_aduana = ttl_to_dataframe_generic(ruta_ttl_aduana,propiedades_aduana,prefijo)
df_ttl_aduana.head()

,IDENTIFICADOR_ADUANA,NOMBRE_ADUANA_TTL
0,aduana55,TALCAHUANO
1,aduana83,COYHAIQUE
2,aduana14,ANTOFAGASTA
3,aduana34,VALPARAISO
4,aduana39,SAN ANTONIO


In [7]:
# Cargar archivo de Importadores
ruta_ttl_importador = "https://deinsoluciones.cl/web_semantica/importadores.ttl"
prefijo = "IMPORTADOR"
propiedades_importadores = {
    "http://schema.org/comuna_importador": "COMUNA_IMPORTADOR",
    "http://schema.org/numero_unico_importador": "NUMERO_IMPORTADOR"
}
df_ttl_importador = ttl_to_dataframe_generic(ruta_ttl_importador,propiedades_importadores,prefijo)
df_ttl_importador.head()

,IDENTIFICADOR_IMPORTADOR,NUMERO_IMPORTADOR,COMUNA_IMPORTADOR
0,cliente73,6369,LIMACHE
1,cliente64,12367,CONCEPCION
2,cliente52,898,VITACURA
3,cliente99,11953,RENCA
4,cliente30,13058,EL BOSQUE


In [8]:
# Cargar archivo de Puertos
ruta_ttl_puerto = "https://deinsoluciones.cl/web_semantica/puertos.ttl"
prefijo = "PUERTO"
propiedades_puerto= {
    "http://schema.org/nombre": "NOMBRE_PUERTO_TTL"
}
df_ttl_puerto = ttl_to_dataframe_generic(ruta_ttl_puerto,propiedades_puerto,prefijo)
df_ttl_puerto.head()

,IDENTIFICADOR_PUERTO,NOMBRE_PUERTO_TTL
0,puerto158,BROWNSVILLE
1,puerto956,VISVIRI
2,puerto133,BRIDGEPORT
3,puerto210,OTROS PUERTOS MEXICO
4,puerto589,PORI


In [9]:
# Cargar archivo de Paises
ruta_ttl_pais = "https://deinsoluciones.cl/web_semantica/paises.ttl"
prefijo = "PAIS"
propiedades_pais= {
    "http://schema.org/nombre": "NOMBRE_PAIS_TTL"
}
df_ttl_pais = ttl_to_dataframe_generic(ruta_ttl_pais,propiedades_pais,prefijo)
df_ttl_pais.head()

,IDENTIFICADOR_PAIS,NOMBRE_PAIS_TTL
0,pais113,BOTSWANA
1,pais520,GRECIA
2,pais535,MONACO
3,pais324,PAKISTAN
4,pais205,JAMAICA


In [10]:
# Cargar archivo de Naves
ruta_ttl_nave = "https://deinsoluciones.cl/web_semantica/naves.ttl"
prefijo = "NAVE"
propiedades_nave = {
    "http://schema.org/agente_nave": "AGENTE_NAVE_TTL",
    "http://schema.org/manifiesto": "MANIFIESTO_TTL",
    "http://schema.org/nombre": "NOMBRE_NAVE_TTL",
}
df_ttl_nave = ttl_to_dataframe_generic(ruta_ttl_nave,propiedades_nave,prefijo)
df_ttl_nave.head()

,IDENTIFICADOR_NAVE,MANIFIESTO_TTL,AGENTE_NAVE_TTL,NOMBRE_NAVE_TTL
0,nave73,235359,INCHCAPESHIPPING SERVICES B.V. CHILE LIMITADA,SIRIUS LEADER
1,nave23,238107,AGENCIAS UNIVERSALES S.A.,CARRERA
2,nave61,238125,ULTRAMAR AGENCIA MARITIMA LIMITADA,LEO SPIRIT
3,nave46,235746,nan,nan
4,nave59,238735,AGENCIAS UNIVERSALES S.A.,SEASPAN BRAVO


In [11]:
df_consolidado = df_consolidado.merge(df_ttl_aduana, left_on='NOMBRE_ADUANA', right_on='NOMBRE_ADUANA_TTL', how='left', suffixes=('', '_ADUANA'))

In [12]:
df_consolidado['NUM_UNICO_IMPORTADOR'] = df_consolidado['NUM_UNICO_IMPORTADOR'].astype(int)
df_ttl_importador['NUMERO_IMPORTADOR'] = df_ttl_importador['NUMERO_IMPORTADOR'].astype(int)
df_consolidado = df_consolidado.merge(df_ttl_importador, left_on='NUM_UNICO_IMPORTADOR', right_on='NUMERO_IMPORTADOR', how='left', suffixes=('', '_IMPORTADOR'))

In [13]:
df_consolidado = df_consolidado.merge(df_ttl_puerto, left_on='NOMBRE_PUERTO_EMBARQUE', right_on='NOMBRE_PUERTO_TTL', how='left', suffixes=('', '_PUERTO_EMB'))
df_consolidado = df_consolidado.merge(df_ttl_puerto, left_on='NOMBRE_PUERTO_DESEMBARQUE', right_on='NOMBRE_PUERTO_TTL', how='left', suffixes=('', '_PUERTO_DES'))

In [14]:
df_consolidado = df_consolidado.merge(df_ttl_pais, left_on='NOMBRE_PAIS_ORIG', right_on='NOMBRE_PAIS_TTL', how='left', suffixes=('', '_PAIS_ORI'))


In [15]:
df_consolidado = df_consolidado.merge(df_ttl_nave, left_on='NUM_MANIF', right_on='MANIFIESTO_TTL', how='left', suffixes=('', '_NAVE'))


In [16]:
df_consolidado.insert(0, 'IMPORTACION_ID', range(1, len(df_consolidado) + 1))

In [17]:
df_formato_final = df_consolidado[['IMPORTACION_ID','NUMENCRIPTADO','ARANC_NAC','TOTAL_BULTOS','FOB','IDENTIFICADOR_IMPORTADOR','IDENTIFICADOR_ADUANA','IDENTIFICADOR_PAIS','IDENTIFICADOR_PUERTO','IDENTIFICADOR_PUERTO_PUERTO_DES','IDENTIFICADOR_NAVE']]

In [18]:
df_formato_final.head()

,IMPORTACION_ID,NUMENCRIPTADO,ARANC_NAC,TOTAL_BULTOS,FOB,IDENTIFICADOR_IMPORTADOR,IDENTIFICADOR_ADUANA,IDENTIFICADOR_PAIS,IDENTIFICADOR_PUERTO,IDENTIFICADOR_PUERTO_PUERTO_DES,IDENTIFICADOR_NAVE
0,1,20955958,87024090,4,309000,cliente1,aduana39,pais336,puerto413,puerto906,nave1
1,2,20981434,87021019,1,22437,cliente2,aduana39,pais333,puerto423,puerto906,nave2
2,3,21014856,87021019,1,22437,cliente2,aduana39,pais333,puerto423,puerto906,nave2
3,4,20985879,87021019,1,22437,cliente2,aduana39,pais333,puerto423,puerto906,nave3
4,5,20993803,87021019,26,292031,cliente2,aduana39,pais333,puerto423,puerto906,nave4


In [19]:
def generar_rdf_ttl(df, ruta_salida):
    """
    Genera un archivo RDF/Turtle a partir de un DataFrame.

    :param df: DataFrame con las columnas requeridas.
    :param ruta_salida: Ruta del archivo RDF/Turtle de salida.
    """
    # Definir los prefijos para el archivo Turtle
    prefijos = """@prefix :       <http://example.org/> .
@prefix xsd:    <http://www.w3.org/2001/XMLSchema#> .
@prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix schema: <http://schema.org/> .

"""
    # Lista donde almacenaremos las líneas del archivo TTL
    lineas = [prefijos]

    # Iterar sobre cada fila del DataFrame
    for _, row in df.iterrows():
        # Crear el encabezado de la importación
        importacion_id = f":importacion{row['IMPORTACION_ID']}"
        lineas.append(f"{importacion_id} rdf:type :Importacion ;")

        # Añadir las propiedades básicas
        propiedades = [
            f"    schema:num_encriptado \"{row['NUMENCRIPTADO']}\"^^xsd:integer;",
            f"    schema:arancel_nacional \"{row['ARANC_NAC']}\"^^xsd:string;",
            f"    schema:total_bultos \"{row['TOTAL_BULTOS']}\"^^xsd:integer;",
            f"    schema:fob \"{str(row['FOB']).replace(',', '.')}\"^^xsd:float;",
            f"    schema:importador :{row['IDENTIFICADOR_IMPORTADOR']};",
            f"    schema:aduana :{row['IDENTIFICADOR_ADUANA']};",
            f"    schema:pais_origen :{row['IDENTIFICADOR_PAIS']};",
            f"    schema:puerto_embarque :{row['IDENTIFICADOR_PUERTO']};",
            f"    schema:puerto_desembarque :{row['IDENTIFICADOR_PUERTO_PUERTO_DES']};",
            f"    schema:nave :{row['IDENTIFICADOR_NAVE']}"
        ]
        lineas.extend(propiedades[:-1])  # Todas las propiedades menos la última terminan con ;
        lineas.append(f"{propiedades[-1]} .")  # La última termina con .

        lineas.append("")  # Línea en blanco para separar registros

    # Guardar las líneas en el archivo de salida
    with open(ruta_salida, "w", encoding="utf-8") as f:
        f.writelines("\n".join(lineas))
    print(f"Archivo RDF/Turtle generado en: {ruta_salida}")


In [20]:
import os

def anexar_rdf_generico(df, propiedades, prefijo_tipo, ruta_salida):
    """
    Anexa tripletas RDF/Turtle de un DataFrame genérico al archivo RDF existente.

    :param df: DataFrame con las columnas relevantes para generar tripletas.
    :param propiedades: Diccionario que mapea propiedades RDF a columnas del DataFrame.
    :param prefijo_tipo: El tipo RDF (por ejemplo, :Aduana, :Importador).
    :param ruta_salida: Ruta del archivo RDF/Turtle existente donde se anexarán los datos.
    """
    # Lista para almacenar las nuevas líneas
    lineas = []

    # Verificar si el archivo ya tiene contenido
    if os.path.exists(ruta_salida) and os.path.getsize(ruta_salida) > 0:
        lineas.append("")  # Agregar un salto de línea si el archivo ya tiene contenido

    # Iterar sobre cada fila del DataFrame
    for _, row in df.iterrows():
        # Crear el identificador del sujeto
        identificador = row.iloc[0]  # Se asume que la primera columna tiene el identificador
        lineas.append(f":{identificador} rdf:type {prefijo_tipo} ;")

        # Añadir las propiedades
        propiedades_tripleta = []
        for propiedad_uri, columna in propiedades.items():
            valor = row[columna]
            if isinstance(valor, int):
                tipo_xsd = "xsd:integer"
            elif isinstance(valor, float):
                tipo_xsd = "xsd:float"
            else:
                tipo_xsd = "xsd:string"

            propiedad = propiedad_uri.replace("http://schema.org/", "schema:")  # Usar prefijo schema:
            propiedades_tripleta.append(f"    {propiedad} \"{valor}\"^^{tipo_xsd}")

        # Añadir las propiedades al archivo
        for i, tripleta in enumerate(propiedades_tripleta):
            if i == len(propiedades_tripleta) - 1:
                # La última propiedad termina con un punto
                lineas.append(f"{tripleta} .")
            else:
                # Todas las demás propiedades terminan con un punto y coma
                lineas.append(f"{tripleta} ;")

        lineas.append("")  # Línea en blanco para separar registros

    # Anexar las líneas al archivo de salida
    with open(ruta_salida, "a", encoding="utf-8") as f:
        f.writelines("\n".join(lineas))
    print(f"Datos anexados con éxito a: {ruta_salida}")




In [21]:
def reemplazar_uris_schema(ruta_archivo):
    """
    Reemplaza las URIs largas de http://schema.org/ por schema: en un archivo RDF/Turtle
    y corrige el prefijo schema mal formado al final.

    :param ruta_archivo: Ruta del archivo RDF/Turtle que se va a procesar.
    """
    with open(ruta_archivo, "r", encoding="utf-8") as f:
        lineas = f.readlines()

    # Reemplazar las URIs largas de <http://schema.org/> por schema:
    lineas_corregidas = []
    for linea in lineas:
        if "<http://schema.org/" in linea:
            linea = linea.replace("<http://schema.org/", "schema:").replace(">", "")
        lineas_corregidas.append(linea)

    # Reemplazar explícitamente el prefijo mal formado
    lineas_corregidas = [
        "@prefix schema: <http://schema.org/> .\n" if linea.strip() == "@prefix schema: schema: ." else linea
        for linea in lineas_corregidas
    ]

    # Sobrescribir el archivo con las líneas corregidas
    with open(ruta_archivo, "w", encoding="utf-8") as f:
        f.writelines(lineas_corregidas)
    print(f"URIs y prefijos corregidos en el archivo: {ruta_archivo}")


In [22]:
def procesar_categorias():
    """
    Procesa todas las categorías (aduanas, importadores, monedas, etc.) y las agrega al archivo RDF/Turtle.
    """
    # Definir las propiedades y DataFrames para cada categoría
    categorias = [
        {"df": df_ttl_aduana, "propiedades": propiedades_aduana, "tipo": ":Aduana"},
        {"df": df_ttl_importador, "propiedades": propiedades_importadores, "tipo": ":Importador"},
        {"df": df_ttl_puerto, "propiedades": propiedades_puerto, "tipo": ":Puerto"},
        {"df": df_ttl_pais, "propiedades": propiedades_pais, "tipo": ":Pais"},
        {"df": df_ttl_nave, "propiedades": propiedades_nave, "tipo": ":Nave"}
    ]

    # Ruta del archivo de salida
    ruta_salida = "importaciones.ttl"

    # Generar el archivo inicial con las importaciones
    generar_rdf_ttl(df_formato_final, ruta_salida)

    # Procesar y anexar cada categoría
    for categoria in categorias:
        anexar_rdf_generico(
            df=categoria["df"],
            propiedades=categoria["propiedades"],
            prefijo_tipo=categoria["tipo"],
            ruta_salida=ruta_salida
        )

    # Reemplazar las URIs largas por los prefijos (schema:)
    reemplazar_uris_schema(ruta_salida)
    print("Procesamiento de todas las categorías completado.")


In [23]:
procesar_categorias()

Archivo RDF/Turtle generado en: importaciones.ttl
Datos anexados con éxito a: importaciones.ttl
Datos anexados con éxito a: importaciones.ttl
Datos anexados con éxito a: importaciones.ttl
Datos anexados con éxito a: importaciones.ttl
Datos anexados con éxito a: importaciones.ttl
URIs y prefijos corregidos en el archivo: importaciones.ttl
Procesamiento de todas las categorías completado.
